# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235928157611                   -0.50    7.0         
  2   -7.250493023112       -1.84       -1.41    1.0   48.2ms
  3   -7.251205141552       -3.15       -2.24    1.0   7.87ms
  4   -7.251280342563       -4.12       -2.19    3.0   10.6ms
  5   -7.251337084190       -4.25       -2.64    1.0   7.96ms
  6   -7.251337219222       -6.87       -3.05    1.0   7.84ms
  7   -7.251338641745       -5.85       -3.47    1.0   8.06ms
  8   -7.251338784176       -6.85       -4.13    1.0   7.77ms
  9   -7.251338794710       -7.98       -4.16    3.0   10.4ms
 10   -7.251338797607       -8.54       -4.53    2.0   9.70ms
 11   -7.251338798586       -9.01       -5.11    1.0   7.91ms
 12   -7.251338798693       -9.97       -5.53    3.0   10.4ms
 13   -7.251338798703      -10.99       -6.23    1.0   8.13ms
 14   -7.251338798704      -12.04       -6.17    3.0   11.1ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07718783681339494
[ Info: Arnoldi iteration step 2: normres = 0.2712424018991417
[ Info: Arnoldi iteration step 3: normres = 0.8685081918518571
[ Info: Arnoldi iteration step 4: normres = 0.4620835711001057
[ Info: Arnoldi iteration step 5: normres = 0.5547335445231032
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.18e-01, 6.12e-02, 4.67e-01, 2.67e-01, 1.06e-02)
[ Info: Arnoldi iteration step 6: normres = 0.301171398048127
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.14e-02, 2.24e-01, 1.25e-01, 1.15e-01, 9.51e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06069123454547228
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (8.84e-04, 1.07e-02, 5.21e-03, 2.37e-02, 4.62e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10737288228490632
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.10e-05